In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

regressors = [
    #LinearRegression(),
    KNeighborsRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
]

df = pd.read_csv('/Users/rohith/Developer/globalec/RGU-Hack/output_datasetAK47.csv')


In [10]:
# One-hot encoding the 'fuelType', 'make', 'model', and 'transmission' columns
df_encoded = pd.get_dummies(df, columns=['make', 'model', 'transmission', 'fueltype'])

# Now df_encoded has the original columns, plus the one-hot encoded columns
# There is no need to concatenate df with dff or apply LabelEncoder on the one-hot encoded dataframe

# You can check the new dataframe to ensure the encoding is correct
print(df_encoded.head())

df_encoded.to_csv('/Users/rohith/Developer/globalec/RGU-Hack/encoded_dataframe.csv', index=False)


   year  price  mileage  tax   mpg  enginesize  maintenancecostyearly  \
0  2020  22500     2500  145  52.3         1.6                 477.34   
1  2017  11750    10326  145  68.9         1.6                 469.22   
2  2019  11699    20350  145  51.4         1.4                 389.31   
3  2019  31299     4831  145  61.4         2.0                 721.93   
4  2019  31450     4568  145  41.5         2.0                 662.21   

   fuel_cost_per_mile  purchase_cost    fuel_cost  ...  model_220  \
0            0.160017          22500  6400.699044  ...      False   
1            0.121465          11750  4858.585776  ...      False   
2            0.153767          11699  6150.665058  ...      False   
3            0.136302          31299  5452.061238  ...      False   
4            0.190448          31450  7617.932145  ...      False   

   transmission_automatic  transmission_manual  transmission_other  \
0                   False                False               False   
1     

In [12]:
# Assuming 'df' is your dataframe and 'one_hot_encoded_df' is the result after one-hot encoding.
one_hot_features = df_encoded.drop(['price', 'year', 'purchase_cost', 'fuel_cost', 'maintenance_cost', 'total_cost_of_ownership'], axis=1).columns.tolist()
X = df_encoded[one_hot_features]
y = df['price']

In [13]:
#intialize the features and target & spltting the data for training and testing 
exclude_columns = ['price', 'year', 'purchase_cost', 'fuel_cost', 'total_cost_of_ownership', 'make', 'model', 'transmission', 'fueltype']
features = [col for col in df.columns if col not in exclude_columns]
target = 'price'

# Prepare the data for training and testing
X = df[features]
y = df[target]

# 1. Check for non-numeric data
for feature in features:
    if df[feature].dtype not in ['int64', 'float64']:
        print(f"Non-numeric data found in feature: {feature}")
# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=0)

print(y_train.shape)
print(y_valid.shape)
print(X_test.shape)
top_5_X = X.head(5)
print(top_5_X)

(30243,)
(10081,)
(10082, 7)
   mileage  tax   mpg  enginesize  maintenancecostyearly  fuel_cost_per_mile  \
0     2500  145  52.3         1.6                 477.34            0.160017   
1    10326  145  68.9         1.6                 469.22            0.121465   
2    20350  145  51.4         1.4                 389.31            0.153767   
3     4831  145  61.4         2.0                 721.93            0.136302   
4     4568  145  41.5         2.0                 662.21            0.190448   

   maintenance_cost  
0           2386.70  
1           2346.10  
2           1946.55  
3           3609.65  
4           3311.05  


In [ ]:
# TODO:   run only for valiidaion of models / dont run for final model
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

head = 7  # Number of models to evaluate

for model in regressors[:head]:
    model.fit(X_train, y_train)  # Train the model on the training data
    y_pred_valid = model.predict(X_valid)  # Predict on the validation data
    y_pred_train = model.predict(X_train)  # Predict on the training data for comparison

    # Print the model and various evaluation metrics
    print(model)
    print("\tExplained variance train:", explained_variance_score(y_train, y_pred_train))
    print("\tExplained variance valid:", explained_variance_score(y_valid, y_pred_valid))
    print()
    print("\tMean absolute error:", mean_absolute_error(y_valid, y_pred_valid))
    print('\tMean Squared Error:', mean_squared_error(y_valid, y_pred_valid))
    print("\tR2 score:", r2_score(y_valid, y_pred_valid))
    print()
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming 'regressors' is the list of models you want to cross-validate
for model in regressors:
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print(f"Model: {model.__class__.__name__}")
    print(f"RMSE scores: {rmse_scores}")
    print(f"Average RMSE: {np.mean(rmse_scores)}")
    print()
    
importances = model.feature_importances_
feature_importances = pd.Series(importances, index=features)
print(feature_importances.sort_values(ascending=False))

KNeighborsRegressor()
	Explained variance train: 0.6236334467645566
	Explained variance valid: 0.4031733749528166

	Mean absolute error: 5018.295982541415
	Mean Squared Error: 50997447.36083721
	R2 score: 0.40309252874825996

RandomForestRegressor()
	Explained variance train: 0.9899882672679332
	Explained variance valid: 0.9298124815599724

	Mean absolute error: 1518.3637501200549
	Mean Squared Error: 5996597.904509898
	R2 score: 0.9298118969373504

DecisionTreeRegressor()
	Explained variance train: 0.9995471572522582
	Explained variance valid: 0.8759439010278163

	Mean absolute error: 1877.216917470652
	Mean Squared Error: 10598944.114485718
	R2 score: 0.8759430274117093



model training:
    Can leverage any regresor ✅

In [19]:
model=RandomForestRegressor()
model.fit(X_train, y_train)  # Train the model on the training data

RandomForestRegressor()

In [18]:
import xgboost as xgb

# Create an instance of the XGBRegressor model
model = xgb.XGBRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Run below code to test with user inputs 
    annual_mileage
    years_of_ownership


In [17]:
import pandas as pd

# Assuming df is your dataset DataFrame and you've already loaded it.
# user inputs
annual_mileage = float(input("Enter the annual mileage: "))
years_of_ownership = int(input("Enter the number of years you plan to own the car: "))
print(annual_mileage)
print(years_of_ownership)

# Calculate annual fuel cost
df['annual_fuel_cost'] = df['fuel_cost_per_mile'] * annual_mileage

# Calculate total fuel cost for the period of ownership
df['total_fuel_cost'] = df['annual_fuel_cost'] * years_of_ownership

# Calculate total maintenance cost for the period of ownership
df['total_maintenance_cost'] = df['maintenancecostyearly'] * years_of_ownership

# Calculate total cost of ownership
df['total_cost_of_ownership'] = df['price'] + df['total_fuel_cost'] + df['total_maintenance_cost']

# Optionally, sort the DataFrame by total cost of ownership to find the cheapest cars
df_sorted = df.sort_values('total_cost_of_ownership').reset_index(drop=True)

# Let's see the top 5 cars with the least total cost of ownership
top_5_cheapest_cars = df_sorted.head(5)
predicted_prices = model.predict(X[features])

df['predicted_price'] = predicted_prices

# Find the top 3 cheapest cars based on predicted price
top_3_predicted_cars = df.nsmallest(5, 'predicted_price')

# Display the results
top_3_predicted_cars




8000.0
5


,make,model,year,price,transmission,mileage,fueltype,tax,mpg,enginesize,maintenancecostyearly,fuel_cost_per_mile,purchase_cost,fuel_cost,maintenance_cost,total_cost_of_ownership,annual_fuel_cost,total_fuel_cost,total_maintenance_cost,predicted_price
23254,ford,fiesta,2004,950,manual,74000,petrol,160,45.6,1.2,736.93,0.173325,950,6932.986491,3684.65,11567.636491,1386.597298,6932.986491,3684.65,1507.42
36703,ford,ka,2008,1294,manual,59750,petrol,160,43.5,1.3,508.79,0.181692,1294,7267.682391,2543.95,11105.632391,1453.536478,7267.682391,2543.95,1944.56
8340,skoda,fabia,2009,1490,manual,72464,petrol,150,47.9,1.2,589.32,0.165002,1490,6600.087349,2946.60,11036.687349,1320.017470,6600.087349,2946.60,1983.73
37738,vauxhall,corsa,2004,1491,manual,64622,petrol,160,44.8,1.2,795.09,0.176420,1491,7056.789821,3975.45,12523.239821,1411.357964,7056.789821,3975.45,1984.37
2418,volkswagen,polo,2009,1990,manual,65024,petrol,150,48.7,1.2,588.57,0.162292,1990,6491.667023,2942.85,11424.517023,1298.333405,6491.667023,2942.85,2076.02
